In [ ]:
import json
import pandas as pd
from collections import defaultdict
import argparse
from datetime import datetime
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

def credit_scorer(json_path):
    df = pd.read_json(json_path)

    # Token decimal mapping
    token_decimals = {
        'USDC': 6,
        'USDT': 6,
        'DAI': 18,
        'WMATIC': 18,
        'WETH': 18,
        'WBTC': 8,
        'WPOL': 18,  
        'AAVE': 18
    }

    wallet_stats = defaultdict(lambda: {
        'total_deposit_usd': 0.0,
        'total_borrow_usd': 0.0,
        'total_repay_usd': 0.0,
        'total_redeem_usd': 0.0,
        'num_liquidations': 0
    })

    for _, row in df.iterrows():
        wallet = row['userWallet']
        action = row['action'].lower()
        data = row['actionData']

        if action == 'liquidationcall':
            wallet_stats[wallet]['num_liquidations'] += 1
            continue  

        #process deposit/borrow/repay/redeem
        symbol = data.get('assetSymbol', '').upper()
        if not symbol or symbol not in token_decimals:
            continue 

        decimals = token_decimals[symbol]

        try:
            amount_raw = float(data.get('amount', 0))
            amount = amount_raw / (10 ** decimals)
            price = float(data.get('assetPriceUSD', 0))
            usd_value = amount * price
        except:
            usd_value = 0

        if action == 'deposit':
            wallet_stats[wallet]['total_deposit_usd'] += usd_value
        elif action == 'borrow':
            wallet_stats[wallet]['total_borrow_usd'] += usd_value
        elif action == 'repay':
            wallet_stats[wallet]['total_repay_usd'] += usd_value
        elif action == 'redeemunderlying':
            wallet_stats[wallet]['total_redeem_usd'] += usd_value

    # Convert to final dataframe
    wallet_df = pd.DataFrame([
        {
            'userWallet': wallet,
            'total_deposit_usd': stats['total_deposit_usd'],
            'total_borrow_usd': stats['total_borrow_usd'],
            'total_repay_usd':stats['total_repay_usd'],
            'repay_ratio': stats['total_repay_usd'] / stats['total_borrow_usd'] if stats['total_borrow_usd'] > 0 else 1.0,
            'total_redeem_usd': stats['total_redeem_usd'],
            'redeem_ratio':stats['total_redeem_usd']/ stats['total_borrow_usd'] if stats['total_borrow_usd'] > 0 else 1.0,
            'num_liquidations': stats['num_liquidations']
        }
        for wallet, stats in wallet_stats.items()
    ])

    features = ['total_deposit_usd', 'total_borrow_usd', 'repay_ratio', 'total_redeem_usd', 'num_liquidations', 'redeem_ratio']

    X = wallet_df[features]

    # Normalize features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    #Define Autoencoder
    input_dim = X_scaled.shape[1]
    encoding_dim = 3

    input_layer = Input(shape=(input_dim,))
    encoded = Dense(8, activation='relu')(input_layer)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    decoded = Dense(8, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    #Train Autoencoder
    autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=32, verbose=0)

    #Encode Data to Latent Space
    X_encoded = encoder.predict(X_scaled)

    #Cluster Encoded Features
    kmeans = KMeans(n_clusters=5, random_state=42)
    wallet_df['cluster'] = kmeans.fit_predict(X_encoded)

    wallet_df['raw_score'] = (
        wallet_df['repay_ratio'] * 500 +                     # up to 500 points for repay ratio
        (1 - wallet_df['num_liquidations'] / 10.0) * 200 +   # penalize liquidation
        (wallet_df['total_deposit_usd'] > 1e5) * 100 +       # bonus for high deposits
        wallet_df['redeem_ratio'].clip(0, 1) * 100  # up to 100 points for redeem ratio
    )

    wallet_df['credit_score'] = wallet_df['raw_score'].clip(0, 1000)

    wallet_df.drop('raw_score', axis=1, inplace=True)

    return wallet_df
    
cred_stats=credit_scorer('file_path.json')